# 1. Importing modules and functions

In [1]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
import chembl_structure_pipeline
from molvs import standardize_smiles
import numpy as np
import pandas as pd
from copy import deepcopy
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split,KFold, StratifiedKFold, GridSearchCV
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import joblib
import pickle
from numpy import savetxt
from IPython.display import HTML
from rdkit.Chem import PandasTools

[05:22:44] Initializing Normalizer


In [2]:
def convert_smi_to_canon_smi(smi):
    
    try:
        canon_smi = Chem.MolToSmiles(Chem.MolFromSmiles(smi),isomericSmiles = False)
    except:
        canon_smi='wrong_smiles'
    return canon_smi

In [3]:
def standart(smi):
    global m
    if smi!='wrong_smiles':
        try:
            smiles=standardize_smiles(smi)
            m = Chem.MolFromSmiles(smi)
        except:
            smi='error kekule'
    else:
        m = 'check the smiles'
    return m

# MORGAN FP_Gradient Boosting Rat oral LD50, mg/kg  

## Load data and curation work set

In [4]:
# Set file path
df_ws=pd.read_csv('rat_oral_LD50_WS.csv')
df_ws

,CAS_Number,SMILES,pLD50,"LD50,mg/kg"
0,626-48-2,Cc1cc(=O)[nH]c(=O)[nH]1,0.291207,64463.0000
1,27849-94-1,CC(CCc1ccc2c(c1)OCO2)NN,0.440660,75449.0000
2,110-54-3,CCCCCC,0.537460,24980.0000
3,1330-92-3,CCCCC(CC)COC(=O)C1=C(C(=O)OCC(CC)CCCC)CCCC1,0.539250,113917.0000
4,57-55-6,CC(O)CO,0.580330,19989.0000
...,...,...,...,...
7707,56073-10-0,O=c1oc2ccccc2c(O)c1C1CC(c2ccc(-c3ccc(Br)cc3)cc...,6.514700,0.1596
7708,130209-82-4,CC(C)OC(=O)CCC/C=C/CC1C(O)CC(O)C1CCC(O)CCc1ccccc1,6.937100,0.0500
7709,83805-11-2,C=C1/C(=C\C=C2/CCCC3(C)C2CCC3C(C)CCCC(O)(C(F)(...,7.099700,0.0417
7710,1746-01-6,Clc1cc2c(cc1Cl)Oc1cc(Cl)c(Cl)cc1O2,7.206800,0.0199


 Convert a SMILES string to canonical SMILES

In [5]:
df_ws1 = deepcopy(df_ws)
df_ws1["SMILES"] = df_ws1.apply(lambda x: convert_smi_to_canon_smi(x.SMILES), axis=1)
df_ws1

,CAS_Number,SMILES,pLD50,"LD50,mg/kg"
0,626-48-2,Cc1cc(=O)[nH]c(=O)[nH]1,0.291207,64463.0000
1,27849-94-1,CC(CCc1ccc2c(c1)OCO2)NN,0.440660,75449.0000
2,110-54-3,CCCCCC,0.537460,24980.0000
3,1330-92-3,CCCCC(CC)COC(=O)C1=C(C(=O)OCC(CC)CCCC)CCCC1,0.539250,113917.0000
4,57-55-6,CC(O)CO,0.580330,19989.0000
...,...,...,...,...
7707,56073-10-0,O=c1oc2ccccc2c(O)c1C1CC(c2ccc(-c3ccc(Br)cc3)cc...,6.514700,0.1596
7708,130209-82-4,CC(C)OC(=O)CCCC=CCC1C(O)CC(O)C1CCC(O)CCc1ccccc1,6.937100,0.0500
7709,83805-11-2,C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)CCCC(O)(C(F)(F)F...,7.099700,0.0417
7710,1746-01-6,Clc1cc2c(cc1Cl)Oc1cc(Cl)c(Cl)cc1O2,7.206800,0.0199


In [6]:
print('Original data: ', len(df_ws), 'molecules')
print('Failed data: ', len(df_ws1[df_ws1['SMILES']=='wrong_smiles']), 'molecules')

Original data:  7712 molecules
Failed data:  0 molecules


In [7]:
index=df_ws1.index[df_ws1['SMILES']=='wrong_smiles'].tolist()
wrong_smiles=df_ws.iloc[index]
wrong_smiles=wrong_smiles.SMILES
number=[x+1 for x in index]
bad_molecules = pd.DataFrame({'No. failed smiles in original set': number, 'SMILES of wrong structure: ': wrong_smiles}, index=None)
bad_molecules = bad_molecules.set_index('No. failed smiles in original set')
bad_molecules

,SMILES of wrong structure:
No. failed smiles in original set,


##  Standardization  for work set

In [8]:
df_ws1["Molecule"] = df_ws1.apply(lambda x: standart(x.SMILES), axis=1)
moldf_ws=df_ws1[df_ws1['SMILES']!='wrong_smiles']
print('Kept data: ', len(moldf_ws), 'molecules')

Kept data:  7712 molecules


In [9]:
moldf_ws

,CAS_Number,SMILES,pLD50,"LD50,mg/kg",Molecule
0,626-48-2,Cc1cc(=O)[nH]c(=O)[nH]1,0.291207,64463.0000,<rdkit.Chem.rdchem.Mol object at 0x0000022369D...
1,27849-94-1,CC(CCc1ccc2c(c1)OCO2)NN,0.440660,75449.0000,<rdkit.Chem.rdchem.Mol object at 0x0000022369D...
2,110-54-3,CCCCCC,0.537460,24980.0000,<rdkit.Chem.rdchem.Mol object at 0x0000022369D...
3,1330-92-3,CCCCC(CC)COC(=O)C1=C(C(=O)OCC(CC)CCCC)CCCC1,0.539250,113917.0000,<rdkit.Chem.rdchem.Mol object at 0x0000022369D...
4,57-55-6,CC(O)CO,0.580330,19989.0000,<rdkit.Chem.rdchem.Mol object at 0x0000022369D...
...,...,...,...,...,...
7707,56073-10-0,O=c1oc2ccccc2c(O)c1C1CC(c2ccc(-c3ccc(Br)cc3)cc...,6.514700,0.1596,<rdkit.Chem.rdchem.Mol object at 0x000002236ED...
7708,130209-82-4,CC(C)OC(=O)CCCC=CCC1C(O)CC(O)C1CCC(O)CCc1ccccc1,6.937100,0.0500,<rdkit.Chem.rdchem.Mol object at 0x000002236ED...
7709,83805-11-2,C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)CCCC(O)(C(F)(F)F...,7.099700,0.0417,<rdkit.Chem.rdchem.Mol object at 0x000002236ED...
7710,1746-01-6,Clc1cc2c(cc1Cl)Oc1cc(Cl)c(Cl)cc1O2,7.206800,0.0199,<rdkit.Chem.rdchem.Mol object at 0x000002236ED...


In [10]:
y_tr=moldf_ws.pLD50
y_tr

0       0.291207
1       0.440660
2       0.537460
3       0.539250
4       0.580330
          ...   
7707    6.514700
7708    6.937100
7709    7.099700
7710    7.206800
7711    7.602600
Name: pLD50, Length: 7712, dtype: float64

In [11]:
moldf_ws=moldf_ws.Molecule

##  Load data and curation test set

In [12]:
df_ts=pd.read_csv('rat_oral_LD50_TS.csv')
df_ts

,CAS_Number,SMILES,pLD50,"LD50,mg/kg"
0,7782-40-3,C,0.017765,15388.8000
1,2842-38-8,OCCNC1CCCCC1,0.572840,38274.0000
2,66257-53-2,NC(=O)C(=O)O,0.624490,21133.0000
3,2173-56-0,CCCCCOC(=O)CCCC,0.686960,35395.0000
4,4726-93-6,O=C1CCCCC(=O)N1,0.750180,22586.0000
...,...,...,...,...
1924,3385-03-3,CC1(C)OC2CC3C4CC(F)C5=CC(=O)C=CC5(C)C4C(O)CC3(...,5.939000,0.4997
1925,2338-29-6,FC(F)(F)c1nc2c(Cl)c(Cl)c(Cl)c(Cl)c2[nH]1,6.121300,0.2435
1926,128606-48-4,CCOP(=S)(OCC)O/C(C)=C/C(=O)OC,6.282400,0.1399
1927,50585-41-6,Brc1cc2c(cc1Br)Oc1cc(Br)c(Br)cc1O2,6.698800,0.0992


 Convert a SMILES string to canonical SMILES

In [13]:
df_ts1 = deepcopy(df_ts)
df_ts1["SMILES"] = df_ts1.apply(lambda x: convert_smi_to_canon_smi(x.SMILES), axis=1)
df_ts1

,CAS_Number,SMILES,pLD50,"LD50,mg/kg"
0,7782-40-3,C,0.017765,15388.8000
1,2842-38-8,OCCNC1CCCCC1,0.572840,38274.0000
2,66257-53-2,NC(=O)C(=O)O,0.624490,21133.0000
3,2173-56-0,CCCCCOC(=O)CCCC,0.686960,35395.0000
4,4726-93-6,O=C1CCCCC(=O)N1,0.750180,22586.0000
...,...,...,...,...
1924,3385-03-3,CC1(C)OC2CC3C4CC(F)C5=CC(=O)C=CC5(C)C4C(O)CC3(...,5.939000,0.4997
1925,2338-29-6,FC(F)(F)c1nc2c(Cl)c(Cl)c(Cl)c(Cl)c2[nH]1,6.121300,0.2435
1926,128606-48-4,CCOP(=S)(OCC)OC(C)=CC(=O)OC,6.282400,0.1399
1927,50585-41-6,Brc1cc2c(cc1Br)Oc1cc(Br)c(Br)cc1O2,6.698800,0.0992


In [14]:
print('Original data: ', len(df_ts), 'molecules')
print('Failed data: ', len(df_ts1[df_ts1['SMILES']=='wrong_smiles']), 'molecules')

Original data:  1929 molecules
Failed data:  0 molecules


In [15]:
index=df_ts1.index[df_ts1['SMILES']=='wrong_smiles'].tolist()
wrong_smiles=df_ts.iloc[index]
wrong_smiles=wrong_smiles.SMILES
number=[x+1 for x in index]
bad_molecules = pd.DataFrame({'No. failed smiles in original set': number, 'SMILES of wrong structure: ': wrong_smiles}, index=None)
bad_molecules = bad_molecules.set_index('No. failed smiles in original set')
bad_molecules

,SMILES of wrong structure:
No. failed smiles in original set,


##  Standardization  for test set

In [16]:
df_ts1["Molecule"] = df_ts1.apply(lambda x: standart(x.SMILES), axis=1)
moldf_ts=df_ts1[df_ts1['SMILES']!='wrong_smiles']
print('Kept data: ', len(moldf_ts), 'molecules')

Kept data:  1929 molecules


In [17]:
moldf_ts

,CAS_Number,SMILES,pLD50,"LD50,mg/kg",Molecule
0,7782-40-3,C,0.017765,15388.8000,<rdkit.Chem.rdchem.Mol object at 0x0000022369B...
1,2842-38-8,OCCNC1CCCCC1,0.572840,38274.0000,<rdkit.Chem.rdchem.Mol object at 0x0000022369B...
2,66257-53-2,NC(=O)C(=O)O,0.624490,21133.0000,<rdkit.Chem.rdchem.Mol object at 0x0000022369B...
3,2173-56-0,CCCCCOC(=O)CCCC,0.686960,35395.0000,<rdkit.Chem.rdchem.Mol object at 0x0000022369B...
4,4726-93-6,O=C1CCCCC(=O)N1,0.750180,22586.0000,<rdkit.Chem.rdchem.Mol object at 0x0000022369B...
...,...,...,...,...,...
1924,3385-03-3,CC1(C)OC2CC3C4CC(F)C5=CC(=O)C=CC5(C)C4C(O)CC3(...,5.939000,0.4997,<rdkit.Chem.rdchem.Mol object at 0x000002236ED...
1925,2338-29-6,FC(F)(F)c1nc2c(Cl)c(Cl)c(Cl)c(Cl)c2[nH]1,6.121300,0.2435,<rdkit.Chem.rdchem.Mol object at 0x000002236ED...
1926,128606-48-4,CCOP(=S)(OCC)OC(C)=CC(=O)OC,6.282400,0.1399,<rdkit.Chem.rdchem.Mol object at 0x000002236ED...
1927,50585-41-6,Brc1cc2c(cc1Br)Oc1cc(Br)c(Br)cc1O2,6.698800,0.0992,<rdkit.Chem.rdchem.Mol object at 0x000002236ED...


In [18]:
y_ts=moldf_ts.pLD50
y_ts

0       0.017765
1       0.572840
2       0.624490
3       0.686960
4       0.750180
          ...   
1924    5.939000
1925    6.121300
1926    6.282400
1927    6.698800
1928    9.541100
Name: pLD50, Length: 1929, dtype: float64

In [19]:
moldf_ts=moldf_ts.Molecule

## Calculation MorganFingerprint for work set

In [20]:
fp_tr = [AllChem.GetMorganFingerprintAsBitVect(m, radius=2,nBits=1024,useFeatures=False,useChirality = False) for m in moldf_ws]

In [21]:
def rdkit_numpy_convert(fp_tr):
    output = []
    for f in fp_tr:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [22]:
from numpy import savetxt
x_tr = rdkit_numpy_convert(fp_tr)

In [23]:
savetxt('Models/MorganFingerprint/x_tr.csv', x_tr, delimiter=',')

In [24]:
x_tr.shape

(7712, 1024)

## Calculation MorganFingerprint for test set

In [25]:
fp_ts = [AllChem.GetMorganFingerprintAsBitVect(m, radius=2,nBits=1024,useFeatures=False,useChirality = False) for m in moldf_ts]

In [26]:
def rdkit_numpy_convert(fp_ts):
    output = []
    for f in fp_ts:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [27]:
x_ts = rdkit_numpy_convert(fp_ts)

In [30]:
x_ts.shape

(1929, 1024)

In [29]:
x_tr = np.array(x_tr, dtype=np.float32)
y_tr = np.array(y_tr, dtype=np.float32)

 # GradientBoostingRegressor model building and validation

In [30]:
seed = 42

In [31]:
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [32]:
param_grid = {'learning_rate': [0.02,0.05],
                  'subsample'    : [0.9, 0.5, 0.1],
                  'n_estimators' : [100,500,1000],
                  'max_depth'    : [4, 10]
                 }

In [33]:
m = GridSearchCV(GradientBoostingRegressor(), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [34]:
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.02, 0.05], 'max_depth': [4, 10],
                         'n_estimators': [100, 500, 1000],
                         'subsample': [0.9, 0.5, 0.1]},
             verbose=1)

In [41]:
best_GBR = m.best_estimator_

In [42]:
m.best_params_

{'learning_rate': 0.02,
 'max_depth': 10,
 'n_estimators': 1000,
 'subsample': 0.5}

In [43]:
y_pred_ws_GBR = best_GBR.predict(x_tr)

In [44]:
R2_WS = round(r2_score(y_tr, y_pred_ws_GBR), 2)
R2_WS

0.92

In [45]:
RMSE_WS=round(np.sqrt(mean_absolute_error(y_tr, y_pred_ws_GBR)), 2)
RMSE_WS

0.45

In [46]:
y_pred_CV_GBR = cross_val_predict(best_GBR, x_tr, y_tr, cv=cv)

In [47]:
y_pred_CV_GBR

array([1.79598986, 2.88487323, 1.85510171, ..., 5.70297225, 2.64026789,
       2.81282546])

In [48]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_GBR), 2)
Q2_CV

0.55

In [49]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_CV_GBR)), 2)
RMSE_CV

0.66

# 9. Prediction for test set's molecules

In [50]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [51]:
y_pred_GBR = best_GBR.predict(x_ts)

In [52]:
Q2_TS = round(r2_score(y_ts, y_pred_GBR), 2)
Q2_TS

0.56

In [53]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts, y_pred_GBR)), 2)
RMSE_TS

0.65

# save the model to disk

In [56]:
pickle.dump(best_GBR, open('models/MorganFingerprint/Toxicity_GBR_MF.pkl', 'wb'))

# load the model from disk

In [21]:
best_GBR = pickle.load(open('Models/Morgan_fingerprint/Toxicity_GBR_MF.pkl', 'rb'))

# 10. Y-randomization GradientBoostingRegressor model

In [57]:
permutations = 50
score, permutation_scores, pvalue = permutation_test_score(best_GBR, x_tr, y_tr,
                                                           cv=cv, scoring='r2',
                                                           n_permutations=permutations,
                                                           n_jobs=-1,
                                                           verbose=1,
                                                           random_state=seed)
print('True score = ', score.round(2),
      '\nY-randomization = ', np.mean(permutation_scores).round(2),
      '\np-value = ', pvalue.round(4))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed: 331.1min


True score =  0.55 
Y-randomization =  -0.15 
p-value =  0.0196


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 541.9min finished


# 11. Estimating applicability domain. Method - Euclidian distances, K=1

In [58]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [59]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,7702,7703,7704,7705,7706,7707,7708,7709,7710,7711
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3.162278,4.000000,1.414214,3.741657,2.236068,3.464102,3.316625,6.000000,0.000000,2.000000,...,2.449490,3.316625,4.000000,4.795832,4.358899,2.645751,6.557438,3.316625,2.645751,5.567764
2,3.464102,4.690416,2.236068,3.872983,2.449490,3.464102,3.605551,6.082763,0.000000,2.449490,...,2.645751,3.464102,4.690416,5.099020,5.477226,4.472136,6.557438,4.242640,2.645751,5.916080
3,3.872983,4.795832,2.449490,3.872983,2.449490,3.464102,3.741657,6.082763,0.000000,2.449490,...,2.828427,4.000000,6.164414,6.164414,5.477226,5.099020,6.633250,5.099020,2.828427,6.244998
4,3.872983,5.099020,2.449490,4.000000,2.449490,3.464102,3.741657,6.082763,1.732051,2.645751,...,3.741657,4.242640,6.244998,6.782330,5.477226,5.099020,6.633250,7.416198,3.316625,6.324555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,10.630146,10.816654,10.583005,10.723805,10.246951,10.344080,10.677078,10.723805,10.677078,10.816654,...,10.954452,11.224972,11.224972,11.916375,11.357817,11.661903,11.224972,11.832160,10.677078,11.489125
7708,11.180340,11.045361,11.224972,11.445523,10.908712,11.090536,10.862781,11.224972,11.313708,11.357817,...,11.045361,11.747340,11.401754,12.206555,11.532562,11.958261,11.618950,12.247449,10.770329,11.832160
7709,11.704700,11.832160,11.747340,12.041595,11.357817,11.445523,11.401754,11.747340,11.747340,11.789826,...,12.000000,12.083046,11.832160,12.767145,12.124355,12.529964,12.206555,12.328828,11.747340,12.727922
7710,11.874342,11.916375,11.747340,12.124355,11.532562,11.532562,11.489125,11.916375,11.832160,11.958261,...,12.000000,12.165525,12.083046,13.000000,12.288206,12.688578,12.369317,12.569805,11.832160,12.884099


In [60]:
similarity= neighbors_k

In [61]:
Dmean=np.mean(similarity[1,:])

In [62]:
round(Dmean, 2)

3.63

In [63]:
std=np.std(similarity[1,:])

In [64]:
round(std, 2)

1.23

In [65]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

4.25


In [66]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [67]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,1919,1920,1921,1922,1923,1924,1925,1926,1927,1928
0,2.000000,3.464102,2.000000,1.414214,2.000000,2.645751,3.162278,0.000000,2.449490,4.358899,...,6.855655,4.690416,2.828427,3.741657,3.162278,4.898980,2.645751,4.123106,3.162278,7.071068
1,2.000000,3.464102,2.236068,1.732051,3.162278,2.828427,3.605551,2.449490,2.449490,4.690416,...,7.071068,6.000000,3.316625,3.872983,3.162278,5.916080,2.828427,4.358899,3.605551,7.280110
2,2.000000,3.605551,2.236068,1.732051,3.162278,2.828427,3.872983,2.449490,2.449490,4.795832,...,7.280110,6.782330,3.316625,4.795832,3.162278,6.082763,2.828427,4.582576,3.741657,7.348469
3,2.000000,3.872983,2.645751,2.000000,3.464102,2.828427,4.000000,2.449490,2.449490,5.196152,...,7.280110,6.855655,3.464102,4.795832,3.316625,6.082763,2.828427,4.690416,3.741657,7.348469
4,2.000000,4.000000,2.828427,2.236068,3.605551,2.828427,4.242640,2.449490,2.645751,5.196152,...,7.416198,6.855655,3.605551,4.898980,3.605551,6.480741,3.000000,4.690416,3.872983,7.416198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,10.583005,10.583005,10.295630,10.770329,10.246951,10.344080,11.000000,10.488089,10.535654,10.954452,...,11.789826,11.618950,11.401754,11.180340,11.269427,11.789826,11.357817,11.224972,10.770329,11.489125
7708,11.135529,11.135529,10.954452,11.313708,11.045361,11.000000,11.618950,11.135529,11.000000,11.224972,...,12.124355,12.124355,11.661903,11.445523,11.618950,11.874342,11.704700,11.661903,10.954452,12.083046
7709,11.575837,11.661903,11.575837,11.832160,11.575837,11.532562,11.704700,11.661903,11.618950,11.747340,...,12.449900,12.609520,12.165525,12.124355,11.958261,12.206555,12.124355,12.083046,11.832160,12.165525
7710,11.661903,11.661903,11.575837,11.916375,11.661903,11.532562,11.789826,11.661903,11.704700,12.000000,...,12.845233,12.688578,12.247449,12.124355,12.041595,12.369317,12.206555,12.247449,11.832160,12.247449


In [68]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[2.    3.464 2.    ... 4.123 3.162 7.071]


In [69]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True  True ...  True  True False]


In [70]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.74


In [71]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [   0    1    2 ... 1925 1926 1927]


In [72]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# 12. Prediction only for molecules included in  AD

In [73]:
y_pred_GBR_ad=list(y_pred_GBR)

In [74]:
y_pred_GBR_ad[:] = [x for i,x in enumerate(y_pred_GBR_ad) if i not in out_Ad]

In [75]:
len(y_pred_GBR_ad)

1429

In [76]:
y_ts_ad=list(y_ts)

In [77]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [78]:
len(y_ts_ad)

1429

In [79]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_GBR_ad), 2)
Q2_TS

0.65

In [80]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts_ad, y_pred_GBR_ad)), 2)
RMSE_TS

0.63

# SVM model building and validation

In [81]:
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}

In [82]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [83]:
svm = GridSearchCV(SVR(C=1.0, epsilon=0.2), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [84]:
svm.fit(x_tr, y_tr)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=SVR(epsilon=0.2), n_jobs=-1,
             param_grid={'C': [1, 10, 100, 1000, 10000],
                         'gamma': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]},
             verbose=1)

In [85]:
svm.best_params_
best_svm = svm.best_estimator_

In [86]:
svm.best_params_

{'C': 10, 'gamma': 0.01}

In [87]:
y_pred_ws_svm = best_svm.predict(x_tr)

In [88]:
R2_WS = round(r2_score(y_tr, y_pred_ws_svm), 2)
R2_WS

0.87

In [89]:
RMSE_WS=round(np.sqrt(mean_absolute_error(y_tr, y_pred_ws_svm)), 2)
RMSE_WS

0.48

In [90]:
y_pred_CV_svm = cross_val_predict(best_svm, x_tr, y_tr, cv=cv)

In [91]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_svm), 2)
Q2_CV

0.5

In [92]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_CV_svm)), 2)
RMSE_CV

0.68

# 9. Prediction for test set's molecules

In [93]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [94]:
y_pred_svm = best_svm.predict(x_ts)

In [95]:
Q2_TS = round(r2_score(y_ts, y_pred_svm), 2)
Q2_TS

0.55

In [96]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts, y_pred_svm)), 2)
RMSE_TS

0.66

save the model to disk

In [99]:
pickle.dump(best_svm, open('models/MorganFingerprint/Toxicity_SVM_MF.pkl', 'wb'))

load the model from disk

In [53]:
best_svm = pickle.load(open('Models/Morgan_fingerprint/Toxicity_SVM_MF.pkl', 'rb'))

# 10. Y-randomization SVM model

In [100]:
permutations = 50
score, permutation_scores, pvalue = permutation_test_score(best_svm, x_tr, y_tr,
                                                           cv=cv, scoring='r2',
                                                           n_permutations=permutations,
                                                           n_jobs=-1,
                                                           verbose=1,
                                                           random_state=seed)
print('True score = ', score.round(3),
      '\nY-randomization = ', np.mean(permutation_scores).round(2),
      '\np-value = ', pvalue.round(4))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed: 39.1min


True score =  0.502 
Y-randomization =  -0.2 
p-value =  0.0196


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 62.6min finished


# 11. Estimating applicability domain. Method - Euclidian distances, K=1

In [101]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [102]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,7702,7703,7704,7705,7706,7707,7708,7709,7710,7711
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3.162278,4.000000,1.414214,3.741657,2.236068,3.464102,3.316625,6.000000,0.000000,2.000000,...,2.449490,3.316625,4.000000,4.795832,4.358899,2.645751,6.557438,3.316625,2.645751,5.567764
2,3.464102,4.690416,2.236068,3.872983,2.449490,3.464102,3.605551,6.082763,0.000000,2.449490,...,2.645751,3.464102,4.690416,5.099020,5.477226,4.472136,6.557438,4.242640,2.645751,5.916080
3,3.872983,4.795832,2.449490,3.872983,2.449490,3.464102,3.741657,6.082763,0.000000,2.449490,...,2.828427,4.000000,6.164414,6.164414,5.477226,5.099020,6.633250,5.099020,2.828427,6.244998
4,3.872983,5.099020,2.449490,4.000000,2.449490,3.464102,3.741657,6.082763,1.732051,2.645751,...,3.741657,4.242640,6.244998,6.782330,5.477226,5.099020,6.633250,7.416198,3.316625,6.324555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,10.630146,10.816654,10.583005,10.723805,10.246951,10.344080,10.677078,10.723805,10.677078,10.816654,...,10.954452,11.224972,11.224972,11.916375,11.357817,11.661903,11.224972,11.832160,10.677078,11.489125
7708,11.180340,11.045361,11.224972,11.445523,10.908712,11.090536,10.862781,11.224972,11.313708,11.357817,...,11.045361,11.747340,11.401754,12.206555,11.532562,11.958261,11.618950,12.247449,10.770329,11.832160
7709,11.704700,11.832160,11.747340,12.041595,11.357817,11.445523,11.401754,11.747340,11.747340,11.789826,...,12.000000,12.083046,11.832160,12.767145,12.124355,12.529964,12.206555,12.328828,11.747340,12.727922
7710,11.874342,11.916375,11.747340,12.124355,11.532562,11.532562,11.489125,11.916375,11.832160,11.958261,...,12.000000,12.165525,12.083046,13.000000,12.288206,12.688578,12.369317,12.569805,11.832160,12.884099


In [103]:
similarity= neighbors_k

In [104]:
Dmean=np.mean(similarity[1,:])

In [105]:
round(Dmean, 2)

3.63

In [106]:
std=np.std(similarity[1,:])

In [107]:
round(std, 2)

1.23

In [108]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

4.25


In [109]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [110]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,1919,1920,1921,1922,1923,1924,1925,1926,1927,1928
0,2.000000,3.464102,2.000000,1.414214,2.000000,2.645751,3.162278,0.000000,2.449490,4.358899,...,6.855655,4.690416,2.828427,3.741657,3.162278,4.898980,2.645751,4.123106,3.162278,7.071068
1,2.000000,3.464102,2.236068,1.732051,3.162278,2.828427,3.605551,2.449490,2.449490,4.690416,...,7.071068,6.000000,3.316625,3.872983,3.162278,5.916080,2.828427,4.358899,3.605551,7.280110
2,2.000000,3.605551,2.236068,1.732051,3.162278,2.828427,3.872983,2.449490,2.449490,4.795832,...,7.280110,6.782330,3.316625,4.795832,3.162278,6.082763,2.828427,4.582576,3.741657,7.348469
3,2.000000,3.872983,2.645751,2.000000,3.464102,2.828427,4.000000,2.449490,2.449490,5.196152,...,7.280110,6.855655,3.464102,4.795832,3.316625,6.082763,2.828427,4.690416,3.741657,7.348469
4,2.000000,4.000000,2.828427,2.236068,3.605551,2.828427,4.242640,2.449490,2.645751,5.196152,...,7.416198,6.855655,3.605551,4.898980,3.605551,6.480741,3.000000,4.690416,3.872983,7.416198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,10.583005,10.583005,10.295630,10.770329,10.246951,10.344080,11.000000,10.488089,10.535654,10.954452,...,11.789826,11.618950,11.401754,11.180340,11.269427,11.789826,11.357817,11.224972,10.770329,11.489125
7708,11.135529,11.135529,10.954452,11.313708,11.045361,11.000000,11.618950,11.135529,11.000000,11.224972,...,12.124355,12.124355,11.661903,11.445523,11.618950,11.874342,11.704700,11.661903,10.954452,12.083046
7709,11.575837,11.661903,11.575837,11.832160,11.575837,11.532562,11.704700,11.661903,11.618950,11.747340,...,12.449900,12.609520,12.165525,12.124355,11.958261,12.206555,12.124355,12.083046,11.832160,12.165525
7710,11.661903,11.661903,11.575837,11.916375,11.661903,11.532562,11.789826,11.661903,11.704700,12.000000,...,12.845233,12.688578,12.247449,12.124355,12.041595,12.369317,12.206555,12.247449,11.832160,12.247449


In [111]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[2.    3.464 2.    ... 4.123 3.162 7.071]


In [112]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True  True ...  True  True False]


In [113]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.74


In [114]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [   0    1    2 ... 1925 1926 1927]


In [115]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# 12. Prediction only for molecules included in  AD

In [116]:
y_pred_svm_ad=list(y_pred_svm)

In [117]:
y_pred_svm_ad[:] = [x for i,x in enumerate(y_pred_svm_ad) if i not in out_Ad]

In [118]:
len(y_pred_svm_ad)

1429

In [119]:
y_ts_ad=list(y_ts)

In [120]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [121]:
len(y_ts_ad)

1429

In [122]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_svm_ad), 2)
Q2_TS

0.64

In [123]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts_ad, y_pred_svm_ad)), 2)
RMSE_TS

0.64

# Multi-layer Perceptron regressor

In [124]:
from sklearn.neural_network import MLPRegressor

In [125]:
param_grid ={"hidden_layer_sizes": [(400, 300, 200, 100),(100, 100, 100), (10, 10, 10),(50,)], "activation": ["tanh", "relu"], "solver": ["lbfgs", "sgd", "adam"], "alpha": [0.00005,0.0005], 'max_iter': [1000, 2000]}

In [126]:
m = GridSearchCV(MLPRegressor(), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [127]:
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=MLPRegressor(), n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'],
                         'alpha': [5e-05, 0.0005],
                         'hidden_layer_sizes': [(400, 300, 200, 100),
                                                (100, 100, 100), (10, 10, 10),
                                                (50,)],
                         'max_iter': [1000, 2000],
                         'solver': ['lbfgs', 'sgd', 'adam']},
             verbose=1)

In [128]:
best_MLPR = m.best_estimator_

In [129]:
m.best_params_

{'activation': 'relu',
 'alpha': 0.0005,
 'hidden_layer_sizes': (400, 300, 200, 100),
 'max_iter': 2000,
 'solver': 'adam'}

In [130]:
y_pred_ws_MLPR = best_MLPR.predict(x_tr)

In [131]:
R2_WS = round(r2_score(y_tr, y_pred_ws_MLPR), 2)
R2_WS

0.99

In [132]:
RMSE_WS=round(np.sqrt(mean_absolute_error(y_tr, y_pred_ws_MLPR)), 2)
RMSE_WS

0.25

In [133]:
y_pred_CV_MLPR = cross_val_predict(best_MLPR, x_tr, y_tr, cv=cv)

In [134]:
y_pred_CV_MLPR

array([1.2884848, 2.330556 , 1.6006368, ..., 5.5600123, 3.146596 ,
       2.9861572], dtype=float32)

In [135]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_MLPR), 2)
Q2_CV

0.5

In [136]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_CV_MLPR)), 2)
RMSE_CV

0.68

# 9. Prediction for test set's molecules

In [32]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [33]:
y_pred_MLPR = best_MLPR.predict(x_ts)

In [34]:
Q2_TS = round(r2_score(y_ts, y_pred_MLPR), 2)
Q2_TS

0.55

In [35]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts, y_pred_MLPR)), 2)
RMSE_TS

0.65

# save the model to disk

In [141]:
pickle.dump(best_MLPR, open('models/MorganFingerprint/Toxicity_MLPR_MF.pkl', 'wb'))

# load the model from disk

In [31]:
best_MLPR = pickle.load(open('models/MorganFingerprint/Toxicity_MLPR_MF.pkl', 'rb'))

# 10. Y-randomization MLPR

In [142]:
permutations = 50
score, permutation_scores, pvalue = permutation_test_score(best_MLPR, x_tr, y_tr,
                                                           cv=cv, scoring='r2',
                                                           n_permutations=permutations,
                                                           n_jobs=-1,
                                                           verbose=1,
                                                           random_state=seed)
print('True score = ', score.round(2),
      '\nY-randomization = ', np.mean(permutation_scores).round(2),
      '\np-value = ', pvalue.round(4))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed: 32.4min


True score =  0.51 
Y-randomization =  -0.33 
p-value =  0.0196


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 60.9min finished


# 11. Estimating applicability domain. Method - Euclidian distances, K=1

In [36]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [37]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,7702,7703,7704,7705,7706,7707,7708,7709,7710,7711
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3.162278,4.000000,1.414214,3.741657,2.236068,3.464102,3.316625,6.000000,0.000000,2.000000,...,2.449490,3.316625,4.000000,4.795832,4.358899,2.645751,6.557438,3.316625,2.645751,5.567764
2,3.464102,4.690416,2.236068,3.872983,2.449490,3.464102,3.605551,6.082763,0.000000,2.449490,...,2.645751,3.464102,4.690416,5.099020,5.477226,4.472136,6.557438,4.242640,2.645751,5.916080
3,3.872983,4.795832,2.449490,3.872983,2.449490,3.464102,3.741657,6.082763,0.000000,2.449490,...,2.828427,4.000000,6.164414,6.164414,5.477226,5.099020,6.633250,5.099020,2.828427,6.244998
4,3.872983,5.099020,2.449490,4.000000,2.449490,3.464102,3.741657,6.082763,1.732051,2.645751,...,3.741657,4.242640,6.244998,6.782330,5.477226,5.099020,6.633250,7.416198,3.316625,6.324555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,10.630146,10.816654,10.583005,10.723805,10.246951,10.344080,10.677078,10.723805,10.677078,10.816654,...,10.954452,11.224972,11.224972,11.916375,11.357817,11.661903,11.224972,11.832160,10.677078,11.489125
7708,11.180340,11.045361,11.224972,11.445523,10.908712,11.090536,10.862781,11.224972,11.313708,11.357817,...,11.045361,11.747340,11.401754,12.206555,11.532562,11.958261,11.618950,12.247449,10.770329,11.832160
7709,11.704700,11.832160,11.747340,12.041595,11.357817,11.445523,11.401754,11.747340,11.747340,11.789826,...,12.000000,12.083046,11.832160,12.767145,12.124355,12.529964,12.206555,12.328828,11.747340,12.727922
7710,11.874342,11.916375,11.747340,12.124355,11.532562,11.532562,11.489125,11.916375,11.832160,11.958261,...,12.000000,12.165525,12.083046,13.000000,12.288206,12.688578,12.369317,12.569805,11.832160,12.884099


In [38]:
similarity= neighbors_k

In [39]:
Dmean=np.mean(similarity[1,:])

In [40]:
round(Dmean, 2)

3.63

In [41]:
std=np.std(similarity[1,:])

In [42]:
round(std, 2)

1.23

In [43]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

4.25


In [44]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [45]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,1919,1920,1921,1922,1923,1924,1925,1926,1927,1928
0,2.000000,3.464102,2.000000,1.414214,2.000000,2.645751,3.162278,0.000000,2.449490,4.358899,...,6.855655,4.690416,2.828427,3.741657,3.162278,4.898980,2.645751,4.123106,3.162278,7.071068
1,2.000000,3.464102,2.236068,1.732051,3.162278,2.828427,3.605551,2.449490,2.449490,4.690416,...,7.071068,6.000000,3.316625,3.872983,3.162278,5.916080,2.828427,4.358899,3.605551,7.280110
2,2.000000,3.605551,2.236068,1.732051,3.162278,2.828427,3.872983,2.449490,2.449490,4.795832,...,7.280110,6.782330,3.316625,4.795832,3.162278,6.082763,2.828427,4.582576,3.741657,7.348469
3,2.000000,3.872983,2.645751,2.000000,3.464102,2.828427,4.000000,2.449490,2.449490,5.196152,...,7.280110,6.855655,3.464102,4.795832,3.316625,6.082763,2.828427,4.690416,3.741657,7.348469
4,2.000000,4.000000,2.828427,2.236068,3.605551,2.828427,4.242640,2.449490,2.645751,5.196152,...,7.416198,6.855655,3.605551,4.898980,3.605551,6.480741,3.000000,4.690416,3.872983,7.416198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,10.583005,10.583005,10.295630,10.770329,10.246951,10.344080,11.000000,10.488089,10.535654,10.954452,...,11.789826,11.618950,11.401754,11.180340,11.269427,11.789826,11.357817,11.224972,10.770329,11.489125
7708,11.135529,11.135529,10.954452,11.313708,11.045361,11.000000,11.618950,11.135529,11.000000,11.224972,...,12.124355,12.124355,11.661903,11.445523,11.618950,11.874342,11.704700,11.661903,10.954452,12.083046
7709,11.575837,11.661903,11.575837,11.832160,11.575837,11.532562,11.704700,11.661903,11.618950,11.747340,...,12.449900,12.609520,12.165525,12.124355,11.958261,12.206555,12.124355,12.083046,11.832160,12.165525
7710,11.661903,11.661903,11.575837,11.916375,11.661903,11.532562,11.789826,11.661903,11.704700,12.000000,...,12.845233,12.688578,12.247449,12.124355,12.041595,12.369317,12.206555,12.247449,11.832160,12.247449


In [46]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[2.    3.464 2.    ... 4.123 3.162 7.071]


In [47]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True  True ...  True  True False]


In [48]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.74


In [49]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [   0    1    2 ... 1925 1926 1927]


In [50]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# 12. Prediction only for molecules included in  AD

In [51]:
y_pred_MLPR_ad=list(y_pred_MLPR)

In [52]:
y_pred_MLPR_ad[:] = [x for i,x in enumerate(y_pred_MLPR_ad) if i not in out_Ad]

In [53]:
len(y_pred_MLPR_ad)

1429

In [54]:
y_ts_ad=list(y_ts)

In [55]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [56]:
len(y_ts_ad)

1429

In [57]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_MLPR_ad), 2)
Q2_TS

0.64

In [58]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts_ad, y_pred_MLPR_ad)), 2)
RMSE_TS

0.64

In [145]:
from sklearn.neighbors import KNeighborsRegressor

# k-nearest neighbors

In [146]:
k_range = list(range(1, 31))
param_grid = dict(n_neighbors=k_range)

In [147]:
m = GridSearchCV(KNeighborsRegressor(), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [148]:
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=KNeighborsRegressor(), n_jobs=-1,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30]},
             verbose=1)

In [149]:
best_kNN = m.best_estimator_

In [150]:
m.best_params_

{'n_neighbors': 4}

In [151]:
y_pred_ws_kNN = best_kNN.predict(x_tr)

In [152]:
R2_WS = round(r2_score(y_tr, y_pred_ws_kNN), 2)
R2_WS

0.66

In [153]:
RMSE_WS=round(np.sqrt(mean_absolute_error(y_tr, y_pred_ws_kNN)), 2)
RMSE_WS

0.6

In [154]:
y_pred_CV_kNN = cross_val_predict(best_kNN, x_tr, y_tr, cv=cv)

In [155]:
y_pred_CV_kNN

array([1.4797826, 2.0088425, 1.4647951, ..., 4.469845 , 2.9031074,
       2.578465 ], dtype=float32)

In [156]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_kNN), 2)
Q2_CV

0.37

In [157]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_CV_kNN)), 2)
RMSE_CV

0.7

# 9. Prediction for test set's molecules

In [158]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [159]:
y_pred_kNN = best_kNN.predict(x_ts)

In [160]:
Q2_TS = round(r2_score(y_ts, y_pred_kNN), 2)
Q2_TS

0.42

In [161]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts, y_pred_kNN)), 2)
RMSE_TS

0.69

# save the model to disk

In [163]:
pickle.dump(best_kNN, open('models/MorganFingerprint/Toxicity_kNN_MF.pkl', 'wb'))

# load the model from disk

In [164]:
best_kNN = pickle.load(open('models/MorganFingerprint/Toxicity_kNN_MF.pkl', 'rb'))

# 10. Y-randomization kNN

In [165]:
permutations = 50
score, permutation_scores, pvalue = permutation_test_score(best_kNN, x_tr, y_tr,
                                                           cv=cv, scoring='r2',
                                                           n_permutations=permutations,
                                                           n_jobs=-1,
                                                           verbose=1,
                                                           random_state=seed)
print('True score = ', score.round(2),
      '\nY-randomization = ', np.mean(permutation_scores).round(2),
      '\np-value = ', pvalue.round(4))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   24.8s


True score =  0.37 
Y-randomization =  -0.25 
p-value =  0.0196


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   42.5s finished


# 11. Estimating applicability domain. Method - Euclidian distances, K=1

In [166]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [167]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,7702,7703,7704,7705,7706,7707,7708,7709,7710,7711
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3.162278,4.000000,1.414214,3.741657,2.236068,3.464102,3.316625,6.000000,0.000000,2.000000,...,2.449490,3.316625,4.000000,4.795832,4.358899,2.645751,6.557438,3.316625,2.645751,5.567764
2,3.464102,4.690416,2.236068,3.872983,2.449490,3.464102,3.605551,6.082763,0.000000,2.449490,...,2.645751,3.464102,4.690416,5.099020,5.477226,4.472136,6.557438,4.242640,2.645751,5.916080
3,3.872983,4.795832,2.449490,3.872983,2.449490,3.464102,3.741657,6.082763,0.000000,2.449490,...,2.828427,4.000000,6.164414,6.164414,5.477226,5.099020,6.633250,5.099020,2.828427,6.244998
4,3.872983,5.099020,2.449490,4.000000,2.449490,3.464102,3.741657,6.082763,1.732051,2.645751,...,3.741657,4.242640,6.244998,6.782330,5.477226,5.099020,6.633250,7.416198,3.316625,6.324555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,10.630146,10.816654,10.583005,10.723805,10.246951,10.344080,10.677078,10.723805,10.677078,10.816654,...,10.954452,11.224972,11.224972,11.916375,11.357817,11.661903,11.224972,11.832160,10.677078,11.489125
7708,11.180340,11.045361,11.224972,11.445523,10.908712,11.090536,10.862781,11.224972,11.313708,11.357817,...,11.045361,11.747340,11.401754,12.206555,11.532562,11.958261,11.618950,12.247449,10.770329,11.832160
7709,11.704700,11.832160,11.747340,12.041595,11.357817,11.445523,11.401754,11.747340,11.747340,11.789826,...,12.000000,12.083046,11.832160,12.767145,12.124355,12.529964,12.206555,12.328828,11.747340,12.727922
7710,11.874342,11.916375,11.747340,12.124355,11.532562,11.532562,11.489125,11.916375,11.832160,11.958261,...,12.000000,12.165525,12.083046,13.000000,12.288206,12.688578,12.369317,12.569805,11.832160,12.884099


In [168]:
similarity= neighbors_k

In [169]:
Dmean=np.mean(similarity[1,:])

In [170]:
round(Dmean, 2)

3.63

In [171]:
std=np.std(similarity[1,:])

In [172]:
round(std, 2)

1.23

In [173]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

4.25


In [174]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [175]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,1919,1920,1921,1922,1923,1924,1925,1926,1927,1928
0,2.000000,3.464102,2.000000,1.414214,2.000000,2.645751,3.162278,0.000000,2.449490,4.358899,...,6.855655,4.690416,2.828427,3.741657,3.162278,4.898980,2.645751,4.123106,3.162278,7.071068
1,2.000000,3.464102,2.236068,1.732051,3.162278,2.828427,3.605551,2.449490,2.449490,4.690416,...,7.071068,6.000000,3.316625,3.872983,3.162278,5.916080,2.828427,4.358899,3.605551,7.280110
2,2.000000,3.605551,2.236068,1.732051,3.162278,2.828427,3.872983,2.449490,2.449490,4.795832,...,7.280110,6.782330,3.316625,4.795832,3.162278,6.082763,2.828427,4.582576,3.741657,7.348469
3,2.000000,3.872983,2.645751,2.000000,3.464102,2.828427,4.000000,2.449490,2.449490,5.196152,...,7.280110,6.855655,3.464102,4.795832,3.316625,6.082763,2.828427,4.690416,3.741657,7.348469
4,2.000000,4.000000,2.828427,2.236068,3.605551,2.828427,4.242640,2.449490,2.645751,5.196152,...,7.416198,6.855655,3.605551,4.898980,3.605551,6.480741,3.000000,4.690416,3.872983,7.416198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,10.583005,10.583005,10.295630,10.770329,10.246951,10.344080,11.000000,10.488089,10.535654,10.954452,...,11.789826,11.618950,11.401754,11.180340,11.269427,11.789826,11.357817,11.224972,10.770329,11.489125
7708,11.135529,11.135529,10.954452,11.313708,11.045361,11.000000,11.618950,11.135529,11.000000,11.224972,...,12.124355,12.124355,11.661903,11.445523,11.618950,11.874342,11.704700,11.661903,10.954452,12.083046
7709,11.575837,11.661903,11.575837,11.832160,11.575837,11.532562,11.704700,11.661903,11.618950,11.747340,...,12.449900,12.609520,12.165525,12.124355,11.958261,12.206555,12.124355,12.083046,11.832160,12.165525
7710,11.661903,11.661903,11.575837,11.916375,11.661903,11.532562,11.789826,11.661903,11.704700,12.000000,...,12.845233,12.688578,12.247449,12.124355,12.041595,12.369317,12.206555,12.247449,11.832160,12.247449


In [176]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[2.    3.464 2.    ... 4.123 3.162 7.071]


In [177]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True  True ...  True  True False]


In [178]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.74


In [179]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [   0    1    2 ... 1925 1926 1927]


In [180]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# 12. Prediction only for molecules included in  AD

In [181]:
y_pred_kNN_ad=list(y_pred_kNN)

In [182]:
y_pred_kNN_ad[:] = [x for i,x in enumerate(y_pred_kNN_ad) if i not in out_Ad]

In [183]:
len(y_pred_kNN_ad)

1429

In [184]:
y_ts_ad=list(y_ts)

In [185]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [186]:
len(y_ts_ad)

1429

In [187]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_kNN_ad), 2)
Q2_TS

0.55

In [188]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts_ad, y_pred_kNN_ad)), 2)
RMSE_TS

0.66